In [95]:
# first neural network with keras tutorial
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [96]:
df = pd.read_pickle('../env_specs/prediction_data.pkl')
df.drop(['ProbeFileID','IsTarget'], axis=1, inplace=True)
# replace nans with -1
df = df.fillna(-1)

In [97]:
X,Y = df.drop(['label'], axis=1).values, df['label'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
# X_train = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12]])

In [98]:
def shuffle(arr):
    x, y = arr.shape
    rows = np.indices((x,y))[0]
    cols = [np.random.permutation(y) for _ in range(x)]
    return arr[rows, cols]

In [99]:
num_samples, num_algos = X_train.shape
del_orders = np.repeat(np.arange(num_algos).reshape(1, num_algos), repeats=num_samples, axis=0)

In [100]:
results = {}

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
h1, h2, h3 = 1000, 1000, 500

# define the keras model
model = Sequential()
model.add(Dense(h1, input_dim=X.shape[1], activation='relu'))
model.add(Dense(h2, input_dim=X.shape[1], activation='relu'))
if h3 > 0:
    model.add(Dense(h3, input_dim=X.shape[1], activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auc])

In [101]:
def del_algos(X, del_orders):
    for i in range(0, X.shape[0], num_algos-1):
        for j in range(num_algos-1):
            X[i+j, del_orders[i//(num_algos-1),:j+1]] = -1.
    return X

In [ ]:
epochs_per_shuffle = 1
X_train_rep = np.repeat(X_train, repeats=num_algos-1, axis=0)
Y_train_rep = np.repeat(Y_train, repeats=num_algos-1, axis=0)

for i in range(50):
    del_orders = shuffle(del_orders)
    X_train_del = del_algos(np.copy(X_train_rep), del_orders)
    
    # fit the keras model on the dataset
    print("Global Epoch: {}".format(i+1))
    model.fit(X_train_del, Y_train_rep, epochs=epochs_per_shuffle, batch_size=30, shuffle=True)
    print("")

Global Epoch: 1
Epoch 1/1
625092/625092 [==============================] - 637s 1ms/step - loss: 0.3392 - acc: 0.8432 - auc: 0.8677

Global Epoch: 2
Epoch 1/1
625092/625092 [==============================] - 637s 1ms/step - loss: 0.2768 - acc: 0.8751 - auc: 0.9224

Global Epoch: 3
Epoch 1/1
625092/625092 [==============================] - 637s 1ms/step - loss: 0.2636 - acc: 0.8797 - auc: 0.9338

Global Epoch: 4
Epoch 1/1
625092/625092 [==============================] - 644s 1ms/step - loss: 0.2562 - acc: 0.8831 - auc: 0.9397

Global Epoch: 5
Epoch 1/1
625092/625092 [==============================] - 648s 1ms/step - loss: 0.2505 - acc: 0.8858 - auc: 0.9432

Global Epoch: 6
Epoch 1/1
625092/625092 [==============================] - 648s 1ms/step - loss: 0.2512 - acc: 0.8851 - auc: 0.9457

Global Epoch: 7
Epoch 1/1
625092/625092 [==============================] - 571s 913us/step - loss: 0.2493 - acc: 0.8864 - auc: 0.9475

Global Epoch: 8
Epoch 1/1
625092/625092 [==========================